In [ ]:
# --- 1. IMPORTAÇÕES ---
import os
from datetime import datetime
import panel as pn
import pandas as pd
import psycopg2 as pg
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Date, DateTime, Text, Numeric, CheckConstraint, or_, Time
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
try: from dotenv import load_dotenv; load_dotenv()
except Exception as e: print(f'Error: {e}')
import hashlib  
try:
    import bcrypt
    _HAS_BCRYPT = True
except Exception:
    _HAS_BCRYPT = False
pn.extension('tabulator', notifications=True, design='material')
pn.config.sizing_mode = 'stretch_width'


In [2]:
# --- 2. ENV ---  
DB_HOST = os.getenv('DB_HOST', '127.0.0.1')
DB_PORT = os.getenv('DB_PORT', '5433')
DB_NAME = os.getenv('DB_NAME', 'Eventos_culturais')  
DB_USER = os.getenv('DB_USER', 'postgres')
DB_PASS = os.getenv('DB_PASS', 'postgres')
def sqlalchemy_url(): return f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'


In [3]:
# --- 3. CONEXÃO ---
_engine = create_engine(sqlalchemy_url(), pool_pre_ping=True)
Session = sessionmaker(bind=_engine)
Base = declarative_base()
cnx = _engine
flag = ''


In [4]:
# --- 4. MODELOS ---
from sqlalchemy.sql import func
class Role(Base):
    __tablename__ = 'role'
    id_papel = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(50), nullable=False, unique=True)
    descricao = Column(Text)

class Usuario(Base):
    __tablename__ = 'usuario'
    id_usuario = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(100), nullable=False)
    email = Column(String(255), nullable=False, unique=True)
    cpf_rg = Column(String(20), nullable=False, unique=True)
    senha_hash = Column(String(255), nullable=False)
    id_papel = Column(Integer, ForeignKey('role.id_papel'), nullable=False)
    role_rel = relationship('Role')
    avaliacoes = relationship('AvaliacaoEvento', back_populates='usuario')

class EspacoCultural(Base):
    __tablename__ = 'espaco_cultural'
    id_espaco_cult = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(100), nullable=False)
    rua = Column(String(100), nullable=False)
    numero = Column(String(10), nullable=False)
    bairro = Column(String(60), nullable=False)
    eventos = relationship('Evento', back_populates='espaco')

class Evento(Base):
    __tablename__ = 'evento'
    id_evento = Column(Integer, primary_key=True, autoincrement=True)
    titulo = Column(String(150), nullable=False)
    descricao = Column(Text)
    categoria = Column(String(60))
    capacidade = Column(Integer)
    data_inicio = Column(DateTime, nullable=False)
    data_fim = Column(DateTime, nullable=False)
    preco = Column(Numeric(10,2), default=0)
    status = Column(String(20), nullable=False)
    id_espaco_cult = Column(Integer, ForeignKey('espaco_cultural.id_espaco_cult'), nullable=False)
    espaco = relationship('EspacoCultural', back_populates='eventos')
    avaliacoes = relationship('AvaliacaoEvento', back_populates='evento')

class AvaliacaoEvento(Base):
    __tablename__ = 'usuario_avalia_evento'
    id_avaliacao = Column(Integer, primary_key=True, autoincrement=True)
    nota = Column(Integer, nullable=False)
    comentario = Column(Text)
    data = Column(DateTime, server_default=func.now(), onupdate=func.now())
    id_usuario = Column(Integer, ForeignKey('usuario.id_usuario'), nullable=False)
    id_evento = Column(Integer, ForeignKey('evento.id_evento'), nullable=False)
    usuario = relationship('Usuario', back_populates='avaliacoes')
    evento = relationship('Evento', back_populates='avaliacoes')

class Denuncia(Base):
    __tablename__ = 'usuario_denuncia_evento'
    id_denuncia = Column(Integer, primary_key=True, autoincrement=True)
    motivo = Column(String(150), nullable=False)
    data = Column(DateTime, nullable=False)
    descricao = Column(Text)
    status = Column(String(20), nullable=False, default='aberta')
    id_usuario = Column(Integer, ForeignKey('usuario.id_usuario'), nullable=False)
    id_evento = Column(Integer, ForeignKey('evento.id_evento'), nullable=False)
    usuario = relationship('Usuario') 
    evento = relationship('Evento')

class Artista(Base):
    __tablename__ = 'artista'
    id_artista = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(100), nullable=False)
    cpf_rg = Column(String(20), unique=True)
    email = Column(String(100))
    telefone = Column(String(20))
    descricao = Column(Text)

# Association Table for Many-to-Many with payload
# Note: In the code it uses raw SQL insert, so we just need the table to exist.
# We can define it as a Table object or a Class. Given usage "artista_apresenta_evento", 
# and raw SQL usage, a Table definition attached to metadata is enough for create_all to pick it up.
from sqlalchemy import Table
artista_apresenta_evento = Table('artista_apresenta_evento', Base.metadata,
    Column('id_artista', Integer, ForeignKey('artista.id_artista'), primary_key=True),
    Column('id_evento', Integer, ForeignKey('evento.id_evento'), primary_key=True),
    Column('data', Date),
    Column('hora', Time)
)


In [ ]:
# --- 5. AUTH ---


CURRENT_USER = None  # None = VISITANTE

def get_role() -> str:
    global CURRENT_USER
    if CURRENT_USER:
         return _norm(CURRENT_USER.get('role', ''))
    return ROLE_VISITANTE


ROLE_ADMIN = "ADMIN"
ROLE_GERENTE = "GERENTE"
ROLE_COMUM = "COMUM"
ROLE_VISITANTE = "VISITANTE"


def _norm(s: str) -> str:
    return (s or "").strip().upper()



# --- Utils (Restored) ---
def format_cpf_live(event):
    # Stub implementation
    pass

def format_tel_live(event):
    # Stub implementation
    pass

def garantir_roles_basicos():
    """
    Garante que existam pelo menos: ADMIN, GERENTE, COMUM
    Rode isso 1x (depois de Base.metadata.create_all).
    """
    s = Session()
    try:
        existentes = { _norm(r.nome) for r in s.query(Role).all() }
        faltando = [ROLE_ADMIN, ROLE_GERENTE, ROLE_COMUM]
        for r in faltando:
            if r not in existentes:
                s.add(Role(nome=r, descricao=f"Papel {r}"))
        s.commit()
    finally:
        s.close()


def get_role_id_by_name(rname: str) -> int:
    """
    Busca id_papel pelo nome (case-insensitive).
    """
    s = Session()
    try:
        r = s.query(Role).filter(Role.nome.ilike(rname.strip())).first()
        if not r:
            # fallback: tenta normalizado
            r = s.query(Role).filter(Role.nome.ilike(_norm(rname))).first()
        if not r:
            raise ValueError(f"Papel '{rname}' não existe na tabela role.")
        return int(r.id_papel)
    finally:
        s.close()


def hash_senha(senha: str) -> str:
   
    
    senha = (senha or "").strip()
    if not senha:
        raise ValueError("Senha não pode ser vazia.")

    if "_HAS_BCRYPT" in globals() and _HAS_BCRYPT:
        hashed = bcrypt.hashpw(senha.encode("utf-8"), bcrypt.gensalt())
        return hashed.decode("utf-8")  # começa com $2b$...
    else:
        return hashlib.sha256(senha.encode("utf-8")).hexdigest()


def verificar_senha(senha_digitada: str, senha_db: str) -> bool:
    """
    Confere senha digitada com o hash salvo no banco.
    """
    if senha_db is None:
        return False

    senha_db = str(senha_db)

   
    if senha_db.startswith("$2"):
        if not ("_HAS_BCRYPT" in globals() and _HAS_BCRYPT):
            raise RuntimeError("Senha no banco está em bcrypt, mas bcrypt não está instalado.")
        return bcrypt.checkpw(
            (senha_digitada or "").encode("utf-8"),
            senha_db.encode("utf-8")
        )

    
    return hashlib.sha256((senha_digitada or "").encode("utf-8")).hexdigest() == senha_db


def tentar_login(email: str, senha: str) -> bool:
    
    global CURRENT_USER
    email = (email or "").strip()
    senha = senha or ""

    s = Session()
    try:
        u = s.query(Usuario).filter(Usuario.email.ilike(email)).first()
        if u and verificar_senha(senha, u.senha_hash):
            r = u.role_rel.nome if u.role_rel else ROLE_COMUM
            CURRENT_USER = {
                "id": int(u.id_usuario),
                "nome": str(u.nome),
                "role": _norm(r),
                "email": str(u.email),
            }
            return True
    finally:
        s.close()

    return False


def cadastrar_usuario_comum(nome: str, email: str, cpf_rg: str, senha: str) -> bool:
    """
    Cadastro público: sempre cria como COMUM e já loga.
    """
    global CURRENT_USER

    nome = (nome or "").strip()
    email = (email or "").strip()
    cpf_rg = (cpf_rg or "").strip()
    senha = senha or ""

    if not nome or not email or not cpf_rg or not senha:
        raise ValueError("Preencha Nome, Email, CPF/RG e Senha.")

    s = Session()
    try:
        # duplicados
        if s.query(Usuario).filter(Usuario.email.ilike(email)).first():
            raise ValueError("Já existe usuário com esse email.")
        if s.query(Usuario).filter(Usuario.cpf_rg == cpf_rg).first():
            raise ValueError("Já existe usuário com esse CPF/RG.")

        rid = get_role_id_by_name(ROLE_COMUM)
        u = Usuario(
            nome=nome,
            email=email,
            cpf_rg=cpf_rg,
            senha_hash=hash_senha(senha),
            id_papel=rid
        )
        s.add(u)
        s.commit()

        # auto-login após cadastro
        return tentar_login(email, senha)
    finally:
        s.close()


def logout():
    global CURRENT_USER
    CURRENT_USER = None


In [6]:
# --- 6. PERMISSÕES E UI DINÂMICA ---

def atualizar_interface():
    role = get_role()
    
    # --- Header ---
    if role == 'VISITANTE':
        header_user.object = 'Modo Visitante'
        btnHeaderLogin.visible = True;
        btnHeaderLogout.visible = False
    else:
        header_user.object = f"Usuário: {CURRENT_USER['nome']} ({get_role()})"
        btnHeaderLogin.visible = False;
        btnHeaderLogout.visible = True
    
    # --- CRUD VISIBILITY ---
    can_edit_content = (role in ['ADMIN', 'GERENTE'])
    row_crud_ev_btns.visible = can_edit_content
    row_crud_esp_btns.visible = can_edit_content
    col_aval_ev.visible = (role == 'COMUM')

    # --- Aba Usuários ---
    can_manage_users = (role in ['ADMIN', 'GERENTE'])
    #tabs_main.objects = [tab_eventos, tab_espacos] if not can_manage_users else [tab_eventos, tab_espacos, tab_usuarios]

    if not can_manage_users:
        tabs_main.objects = [tab_eventos, tab_espacos]
    else:
        try:
            tabs_main.objects = [tab_eventos, tab_espacos, tab_usuarios, tab_denuncias]
        except:
            tabs_main.objects = [tab_eventos, tab_espacos, tab_usuarios]
    
    # OPTIONS UPDATE: UPPERCASE
    if role == 'ADMIN':
        user_role_select.options = ['GERENTE', 'COMUM']
    elif role == 'GERENTE':
        user_role_select.options = ['COMUM']

    # REFRESH VIEWS (Important for Hybrid Views)
    try: on_con_ev()
    except Exception as e: print(f'Error: {e}')
    try: on_con_esp()
    except Exception as e: print(f'Error: {e}')
    try: on_con_users()
    except Exception as e: print(f'Error: {e}')
    try: on_con_art()
    except Exception as e: print(f'Error: {e}')
    try: atualizar_opcoes_categorias()
    except Exception as e: print(f'Error: {e}')


In [7]:
# --- 7. EVENTOS (Full Consolidated) ---
import functools
from sqlalchemy import text

# --- WIDGETS ---
ev_id = pn.widgets.IntInput(name='ID', disabled=True, width=80)
ev_titulo = pn.widgets.TextInput(name='Título')
ev_cat = pn.widgets.Select(name='Categoria', options=[])
ev_status = pn.widgets.Select(name='Status', options=['', 'Agendado', 'Realizado', 'Cancelado'], value='')
ev_dt_ini = pn.widgets.DatePicker(name='Data Início')
ev_dt_fim = pn.widgets.DatePicker(name='Data Fim')

# --- Missing Widgets Restore (Fixed) ---
ev_artistas = pn.widgets.MultiChoice(name='Artistas', option_limit=100)
btn_add_cat = pn.widgets.Button(name='+', width=40, button_type='light')
new_cat_input = pn.widgets.TextInput(placeholder='Nova categoria...')
btn_save_cat = pn.widgets.Button(name='OK', width=40, button_type='success')
btn_cancel_cat = pn.widgets.Button(name='X', width=40, button_type='danger')
row_new_cat = pn.Row(new_cat_input, btn_save_cat, btn_cancel_cat, visible=False)

def atualizar_opcoes_categorias(selected=None):
    s = Session()
    try:
        # Fetch distinct categories
        try: cats = [r[0] for r in s.query(Evento.categoria).distinct().all() if r[0]]
        except: cats = []
        
        if selected and selected not in cats: cats.append(selected)
        current = ev_cat.value
        if current and current not in cats: cats.append(current)
        
        cats.sort()
        cats = [''] + cats
        ev_cat.options = cats
        if selected: ev_cat.value = selected
    except Exception as e: print(f'Error: {e}')
    finally: s.close()

# Handlers for Category Buttons
def on_add_cat_click(e): row_new_cat.visible = True
def on_cancel_cat_click(e): row_new_cat.visible = False; new_cat_input.value=''
def on_save_cat_click(e):
    val = new_cat_input.value
    if val:
        atualizar_opcoes_categorias(selected=val)
        row_new_cat.visible = False
        new_cat_input.value = ''

btn_add_cat.on_click(on_add_cat_click)
btn_cancel_cat.on_click(on_cancel_cat_click)
btn_save_cat.on_click(on_save_cat_click)
atualizar_opcoes_categorias() # Init load
def atualizar_opcoes_artistas():
    s = Session()
    try:
        arts = s.query(Artista).all()
        # MultiChoice options as {Name: ID}
        ev_artistas.options = {a.nome: a.id_artista for a in arts}
    except Exception as e: print(f'Erro ao carregar artistas: {e}')
    finally: s.close()

def carregar_artistas_evento(eid):
    s = Session()
    try:
        # Raw SQL to get IDs (assuming Model not fully mapped for M2M or I missed it)
        # Using text() requires import
        q = text('SELECT id_artista FROM artista_apresenta_evento WHERE id_evento = :eid')
        # We need to ensure we have 'text' imported or use s.connection().execute
        # The notebook imports 'from sqlalchemy import text' in Cell 6/7. So it should be fine.
        res = s.execute(q, {'eid': eid}).fetchall()
        ev_artistas.value = [row[0] for row in res]
    except Exception as e: 
        print(f'Erro ao carregar artistas do evento: {e}')
        ev_artistas.value = []
    finally: s.close()


# Extra Fields
ev_capacidade = pn.widgets.IntInput(name='Capacidade', start=0)
ev_preco = pn.widgets.FloatInput(name='Preço (R$)', start=0.0, step=0.01)
ev_descricao = pn.widgets.TextAreaInput(name='Descrição', height=100)

# Botões CRUD
btnConEv = pn.widgets.Button(name='Consultar', button_type='primary')
btnLimparEv = pn.widgets.Button(name='Limpar', button_type='default')
btnInsEv = pn.widgets.Button(name='Inserir', button_type='success')
btnAttEv = pn.widgets.Button(name='Atualizar', button_type='warning')
btnDelEv = pn.widgets.Button(name='Excluir', button_type='danger')

# Layout Form
col_inputs_ev = pn.Column(
    '### Gestão de Eventos',
    pn.Row(ev_id, ev_status),
    ev_titulo,
    pn.Row(pn.Column(pn.Row(ev_cat, btn_add_cat), row_new_cat), ev_capacidade, ev_preco),
    pn.Row(ev_dt_ini, ev_dt_fim, ev_artistas),
    ev_descricao,
    pn.Row(btnConEv, btnLimparEv)
)
row_crud_ev_btns = pn.Row(btnInsEv, btnAttEv, btnDelEv)
col_ev_form = pn.Column(col_inputs_ev, row_crud_ev_btns)

# Avaliação (Para Comum)
aval_nota = pn.widgets.IntSlider(name='Nota', start=1, end=5, step=1, value=5)
aval_coment = pn.widgets.TextAreaInput(name='Comentário', height=100)
denuncia_motivo = pn.widgets.TextInput(name='Motivo da Denúncia', placeholder='Descreva o motivo da denúncia...')
btnEnviarAval = pn.widgets.Button(name='Avaliar', button_type='primary')
btnDenunciar = pn.widgets.Button(name='Denunciar', button_type='danger')
col_aval_ev = pn.Column(
    '### Interação', 
    aval_nota, 
    btnEnviarAval,
    pn.layout.Divider(),
    denuncia_motivo,
    aval_coment,
    btnDenunciar
)

# --- VISUALIZAÇÃO 1: ADMIN (Tabela Clássica) ---
tab_ev_admin = pn.widgets.Tabulator(pagination='remote', page_size=20, layout='fit_columns', 
                                    selectable=1, show_index=False, disabled=True, 
                                    sizing_mode='stretch_width', theme='bootstrap4')

# --- VISUALIZAÇÃO 2: PÚBLICO (Feed Vertical) ---
btnEvAnt = pn.widgets.Button(name='< Anterior', width=100)
btnEvProx = pn.widgets.Button(name='Próximo >', width=100)
lblEvPage = pn.widgets.StaticText(value='Página 1', align='center')
feed_ev = pn.Column(scroll=True, height=800, sizing_mode='stretch_width')
col_feed_nav = pn.Column(pn.Row(btnEvAnt, lblEvPage, btnEvProx), feed_ev)

ev_page = 0
EV_PAGE_SIZE = 10

# --- LÓGICA ---

def carregar_form(row):
    # Preenche o formulário completo
    try:
        ev_id.value = int(row['id_evento'])
        ev_titulo.value = str(row['titulo'])
        cat_val = str(row['categoria']) if row['categoria'] else ''
        atualizar_opcoes_categorias(cat_val)
        ev_status.value = str(row['status']) if row['status'] else ''
        ev_capacidade.value = int(row['capacidade']) if row['capacidade'] else 0
        ev_preco.value = float(row['preco']) if row['preco'] else 0.0
        ev_descricao.value = str(row['descricao']) if row['descricao'] else ''
        ev_dt_ini.value = pd.to_datetime(row['data_inicio']).date() if row['data_inicio'] else None
        ev_dt_fim.value = pd.to_datetime(row['data_fim']).date() if row['data_fim'] else None
        carregar_artistas_evento(ev_id.value)
        pn.state.notifications.info(f"Carregado: {row['titulo']}")
    except Exception as e: pn.state.notifications.error(f'Erro load: {e}')

# Seleção na Tabela Admin
def on_sel_admin(event):
    if not event.new: return
    try: carregar_form(tab_ev_admin.value.iloc[event.new[0]])
    except Exception as e: print(f'Error: {e}')
tab_ev_admin.param.watch(on_sel_admin, 'selection')

# Card Widget para Público
def criar_widget_publico(row, can_eval):
    dt_str = f"{row['data_inicio'].strftime('%d/%m/%Y')} a {row['data_fim'].strftime('%d/%m/%Y')}" if row['data_inicio'] else "N/D"
    data = {
        'Campo': ['Título', 'Categoria', 'Status', 'Data', 'Local', 'Avaliação', 'Artistas', 'Preço', 'Descrição'],
        'Valor': [
            str(row['titulo']), str(row['categoria']), str(row['status']), dt_str,
            str(row['local']), f"⭐ {row['avaliacao']}", str(row['artistas'] or 'Nenhum'),
            f"R$ {row['preco']}", str(row['descricao'] or '')
        ]
    }
    df_mini = pd.DataFrame(data)
    # Selectable logic for Public
    tbl = pn.widgets.Tabulator(df_mini, show_index=False, disabled=False, 
                               configuration={'headerVisible': False}, 
                               editors={'Campo': None, 'Valor': None}, selectable=1,
                               sizing_mode='stretch_width', theme='bootstrap4')
    
    def on_sel_pub(event):
        if event.new:
            try:
                carregar_form(row)
                try: tbl.selection = []
                except Exception as e: print(f'Error: {e}')
            except Exception as e: print(f'Error: {e}')
    tbl.param.watch(on_sel_pub, 'selection')

    # Aval Button
    btns = []
    if can_eval:
        b = pn.widgets.Button(name='⭐ Avaliar', button_type='primary', width=100)
        b.on_click(lambda e: setattr(ev_id, 'value', int(row['id_evento'])))
        btns.append(b)
    return pn.Column(tbl, pn.Row(*btns), pn.layout.Divider(), margin=(10, 10))

def fetch_events(limit=None, offset=0):
    t = (ev_titulo.value or "").replace("'", "''")
    c = (ev_cat.value or "").replace("'", "''")
    st = ev_status.value
    
    query = f"""
        SELECT 
            e.id_evento, e.titulo, e.descricao, e.categoria, e.capacidade,
            e.data_inicio, e.data_fim, e.preco, e.status,
            ec.nome as "local",
            ROUND(COALESCE(AVG(ae.nota), 0), 1) as avaliacao,
            STRING_AGG(DISTINCT art.nome, ', ') as artistas,
            (
                SELECT COUNT(*)
                FROM usuario_denuncia_evento d
                WHERE d.id_evento = e.id_evento
            ) as qtd_denuncias
        FROM evento e
        JOIN espaco_cultural ec ON e.id_espaco_cult = ec.id_espaco_cult
        LEFT JOIN usuario_avalia_evento ae ON e.id_evento = ae.id_evento
        LEFT JOIN artista_apresenta_evento aae ON e.id_evento = aae.id_evento
        LEFT JOIN artista art ON aae.id_artista = art.id_artista
        WHERE 
            ('{t}'='' OR e.titulo ILIKE '%%{t}%%') 
            AND ('{c}'='' OR e.categoria ILIKE '%%{c}%%')
            AND ('{st}'='' OR e.status = '{st}')
        GROUP BY e.id_evento, ec.nome
        ORDER BY e.data_inicio DESC
    """
    if limit:
        query += f" LIMIT {limit} OFFSET {offset} "
    return pd.read_sql_query(query, cnx)

def on_con_ev(e=None):
    role = get_role()
    is_admin_mode = (role in ['ADMIN', 'GERENTE'])
    
    tab_ev_admin.visible = is_admin_mode
    col_feed_nav.visible = not is_admin_mode
    
    # Input Visibility
    ev_descricao.visible = is_admin_mode
    ev_capacidade.visible = is_admin_mode
    ev_preco.visible = is_admin_mode
    ev_artistas.visible = is_admin_mode

    actual_fetch(is_admin_mode)

def actual_fetch(is_admin):
    if is_admin:
        try: 
             tab_ev_admin.value = fetch_events()
             atualizar_opcoes_artistas()
        except Exception as x: pn.state.notifications.error(f'{x}')
    else:
        # Carrega primeira página do feed
        atualizar_feed_publico()

def atualizar_feed_publico():
    feed_ev.clear()
    feed_ev.append(pn.indicators.LoadingSpinner(value=True, width=30))
    try:
        df = fetch_events(limit=EV_PAGE_SIZE, offset=ev_page * EV_PAGE_SIZE)
        feed_ev.clear()
        if df.empty:
             feed_ev.append(pn.pane.Alert('Fim da lista.', alert_type='warning'))
             return
        
        can_eval = False # (get_role() == 'COMUM')
        for idx, row in df.iterrows():
            feed_ev.append(criar_widget_publico(row, can_eval))
    except Exception as x: pn.state.notifications.error(f'{x}')

def nav_ant(e):
    global ev_page
    if ev_page > 0:
        ev_page -= 1
        lblEvPage.value = f'Página {ev_page + 1}'
        atualizar_feed_publico()

def nav_prox(e):
    global ev_page
    ev_page += 1
    lblEvPage.value = f'Página {ev_page + 1}'
    atualizar_feed_publico()

btnEvAnt.on_click(nav_ant); btnEvProx.on_click(nav_prox)

# --- CRUD LOGIC ---
def on_ins_ev(e=None):
    s = Session()
    try:
        if s.query(Evento).filter_by(titulo=ev_titulo.value).first():
             pn.state.notifications.error('Título duplicado!')
             return
        if not ev_dt_ini.value or not ev_dt_fim.value: 
            raise Exception('Datas obrigatórias')
        
        new_ev = Evento(
            titulo=ev_titulo.value, categoria=ev_cat.value, 
            status=ev_status.value or 'Agendado', 
            data_inicio=ev_dt_ini.value, data_fim=ev_dt_fim.value, 
            id_espaco_cult=1, capacidade=ev_capacidade.value,
            preco=ev_preco.value, descricao=ev_descricao.value)
        s.add(new_ev)
        s.flush() # get ID
        # Insert Artists (Updated Schema: id_evento, id_artista, data, hora)
        dt_val = f"'{ev_dt_ini.value}'" if ev_dt_ini.value else 'NULL'
        for art_id_str in ev_artistas.value:
            s.execute(text(f'INSERT INTO artista_apresenta_evento (id_artista, id_evento, data, hora) VALUES ({art_id_str}, {new_ev.id_evento}, {dt_val}, NULL)'))
        s.commit(); pn.state.notifications.success('Inserido!'); on_con_ev()
    except Exception as x: pn.state.notifications.error(f'{x}')
    finally: s.close()

def on_att_ev(e=None):
    if not ev_id.value: return
    s = Session()
    try:
        o = s.get(Evento, ev_id.value)
        if o: 
            d_ini = ev_dt_ini.value
            d_fim = ev_dt_fim.value
            curr_ini = o.data_inicio.date() if o.data_inicio else None
            curr_fim = o.data_fim.date() if o.data_fim else None
            
            # Check ALL fields
            changed = (
                o.titulo != ev_titulo.value or 
                o.categoria != ev_cat.value or 
                o.status != ev_status.value or 
                curr_ini != d_ini or 
                curr_fim != d_fim or
                o.capacidade != ev_capacidade.value or
                float(o.preco or 0) != ev_preco.value or
                (o.descricao or '') != ev_descricao.value
            )
            
            if not changed:
                pn.state.notifications.warning('Nenhuma alteração detectada')
                return

            o.titulo=ev_titulo.value; o.categoria=ev_cat.value
            if ev_status.value: o.status=ev_status.value
            if d_ini: o.data_inicio=d_ini
            if d_fim: o.data_fim=d_fim
            o.capacidade=ev_capacidade.value
            o.preco=ev_preco.value
            o.descricao=ev_descricao.value
            
            s.commit(); pn.state.notifications.success('Atualizado!'); on_con_ev()
    finally: s.close()

def on_del_ev(e=None):
    s = Session()
    try:
        if o:=s.get(Evento, ev_id.value): s.delete(o); s.commit(); pn.state.notifications.success('Excluído!'); on_con_ev()
    finally: s.close()

def on_limpar_ev(e=None):
    ev_id.value = 0; ev_titulo.value = ''; ev_cat.value = ''; ev_status.value = ''
    ev_dt_ini.value = None; ev_dt_fim.value = None
    ev_capacidade.value = 0; ev_preco.value = 0.0; ev_descricao.value = ''
    ev_artistas.value = []
    tab_ev_admin.selection = []
    on_con_ev()

def on_aval_ev(e=None):
    if not CURRENT_USER: return
    if not ev_id.value: 
        pn.state.notifications.warning('Selecione!')
        return
    s = Session()
    try:
        av = s.query(AvaliacaoEvento).filter_by(id_evento=ev_id.value, id_usuario=CURRENT_USER['id']).first()
        if av:
            av.nota = aval_nota.value
            av.comentario = aval_coment.value 
            pn.state.notifications.success('Avaliação atualizada!')
        else:
            s.add(AvaliacaoEvento(id_evento=ev_id.value, id_usuario=CURRENT_USER['id'], nota=aval_nota.value, comentario=aval_coment.value))
            pn.state.notifications.success('Avaliação enviada!')
        s.commit()
        on_con_ev() # REFRESH FEED
    except Exception as x: pn.state.notifications.error(f'Erro: {x}')
    finally: s.close()
#Teste
def on_denunciar_click(e=None):
    if not CURRENT_USER: 
        pn.state.notifications.error('Faça login para denunciar.')
        return
    
    if not ev_id.value: 
        pn.state.notifications.warning('Selecione um evento para denunciar!')
        return

    if not denuncia_motivo.value:
        pn.state.notifications.warning('Por favor, informe o motivo da denúncia.')
        return

    s = Session()
    try:
        # Cria o objeto Denuncia
        nova_denuncia = Denuncia(
            id_usuario=CURRENT_USER['id'],
            id_evento=ev_id.value,
            motivo=denuncia_motivo.value,
            descricao=aval_coment.value,
            status='aberta',
            data=datetime.now()
        )
        
        s.add(nova_denuncia)
        s.commit()
        
        pn.state.notifications.success('Denúncia enviada com sucesso! A administração irá analisar.')
        
        denuncia_motivo.value = ''
        
        on_con_ev() 
        
    except Exception as x:
        pn.state.notifications.error(f'Erro ao denunciar: {x}')
    finally:
        s.close()    

btnConEv.on_click(on_con_ev); btnLimparEv.on_click(on_limpar_ev)
btnInsEv.on_click(on_ins_ev); btnAttEv.on_click(on_att_ev); btnDelEv.on_click(on_del_ev)
btnEnviarAval.on_click(on_aval_ev); 
btnDenunciar.on_click(on_denunciar_click)

on_con_ev()
tab_eventos = pn.Row(pn.Column(col_ev_form, col_aval_ev), tab_ev_admin, col_feed_nav, name='Eventos')


In [8]:
# --- 8. ESPAÇOS (Hybrid View - NO EVAL) ---

# Widgets CRUD
esp_id = pn.widgets.IntInput(name='ID', disabled=True, width=80)
esp_nome = pn.widgets.TextInput(name='Nome')
esp_rua = pn.widgets.TextInput(name='Rua')
esp_num = pn.widgets.TextInput(name='Número')
esp_bairro = pn.widgets.TextInput(name='Bairro')

btnConEsp = pn.widgets.Button(name='Consultar', button_type='primary')
btnLimparEsp = pn.widgets.Button(name='Limpar', button_type='default')
btnInsEsp = pn.widgets.Button(name='Inserir', button_type='success')
btnAttEsp = pn.widgets.Button(name='Atualizar', button_type='warning')
btnDelEsp = pn.widgets.Button(name='Excluir', button_type='danger')

# Layout Global
col_inputs_esp = pn.Column('### Gestão de Espaços', 
                           esp_id, esp_nome, esp_rua, esp_num, esp_bairro,
                           pn.Row(btnConEsp, btnLimparEsp))

row_crud_esp_btns = pn.Row(btnInsEsp, btnAttEsp, btnDelEsp)
col_esp_main = pn.Column(col_inputs_esp, row_crud_esp_btns)

# --- VIEW 1: ADMIN ---
tab_esp_admin = pn.widgets.Tabulator(pagination='remote', page_size=20, layout='fit_columns', 
                                     selectable=1, show_index=False, disabled=True,
                                     sizing_mode='stretch_width', theme='bootstrap4')

# --- VIEW 2: PUBLIC ---
btnEspAnt = pn.widgets.Button(name='< Anterior', width=100)
btnEspProx = pn.widgets.Button(name='Próximo >', width=100)
lblEspPage = pn.widgets.StaticText(value='Página 1', align='center')
feed_esp = pn.Column(scroll=True, height=800, sizing_mode='stretch_width')
col_feed_esp_nav = pn.Column(pn.Row(btnEspAnt, lblEspPage, btnEspProx), feed_esp)

esp_page = 0
ESP_PAGE_SIZE = 4

# --- LOGIC ---
def carregar_form_esp(row):
    try:
        esp_id.value = int(row['id_espaco_cult'])
        esp_nome.value = str(row['nome'])
        esp_rua.value = str(row['rua']) if row['rua'] else ''
        esp_num.value = str(row['numero']) if row['numero'] else ''
        esp_bairro.value = str(row['bairro']) if row['bairro'] else ''
    except Exception as e: print(f'Error: {e}')

def on_sel_esp_admin(event):
    if not event.new: return
    try:
        row = tab_esp_admin.value.iloc[event.new[0]]
        carregar_form_esp(row)
    except Exception as e: print(f'Error: {e}')
tab_esp_admin.param.watch(on_sel_esp_admin, 'selection')

def criar_widget_espaco_publico(row, can_eval=False):
    data = {
        'Campo': ['Nome', 'Rua', 'Número', 'Bairro'],
        'Valor': [str(row['nome']), str(row['rua']), str(row['numero']), str(row['bairro'])]
    }
    df_mini = pd.DataFrame(data)
    # Updated: Enable Selection
    tbl = pn.widgets.Tabulator(df_mini, show_index=False, disabled=False, 
                               configuration={'headerVisible': False}, 
                               editors={'Campo': None, 'Valor': None}, selectable=1,
                               sizing_mode='stretch_width', theme='bootstrap4')
    
    def on_sel_pub(event):
        if event.new:
            try:
                carregar_form_esp(row)
                try: tbl.selection = []
                except Exception as e: print(f'Error: {e}')
            except Exception as e: print(f'Error: {e}')
    tbl.param.watch(on_sel_pub, 'selection')
    
    return pn.Column(tbl, pn.layout.Divider(), margin=(10, 10))
def fetch_espacos(limit=None, offset=0):
    n = esp_nome.value.replace("'", "''")
    r = esp_rua.value.replace("'", "''")
    b = esp_bairro.value.replace("'", "''")
    
    query = f"""
        SELECT * FROM espaco_cultural 
        WHERE 
            ('{n}'='' OR nome ILIKE '%%{n}%%') 
            AND ('{r}'='' OR rua ILIKE '%%{r}%%')
            AND ('{b}'='' OR bairro ILIKE '%%{b}%%')
        ORDER BY id_espaco_cult
    """
    if limit:
        query += f" LIMIT {limit} OFFSET {offset} "
    return pd.read_sql_query(query, cnx)

def on_con_esp(e=None):
    role = get_role()
    is_admin_mode = (role in ['ADMIN', 'GERENTE'])
    
    tab_esp_admin.visible = is_admin_mode
    col_feed_esp_nav.visible = not is_admin_mode
    row_crud_esp_btns.visible = is_admin_mode
    
    if is_admin_mode:
        try: tab_esp_admin.value = fetch_espacos()
        except Exception as e: print(f'Error: {e}')
    else:
        atualizar_feed_esp_publico()

def atualizar_feed_esp_publico():
    feed_esp.clear()
    feed_esp.append(pn.indicators.LoadingSpinner(value=True, width=30))
    try:
        df = fetch_espacos(limit=ESP_PAGE_SIZE, offset=esp_page * ESP_PAGE_SIZE)
        feed_esp.clear()
        if df.empty:
             feed_esp.append(pn.pane.Alert('Fim da lista.', alert_type='warning'))
             return
        
        for idx, row in df.iterrows():
            feed_esp.append(criar_widget_espaco_publico(row))
    except Exception as x: pn.state.notifications.error(f'{x}')

def nav_esp_ant(e):
    global esp_page
    if esp_page > 0:
        esp_page -= 1
        lblEspPage.value = f'Página {esp_page + 1}'
        atualizar_feed_esp_publico()

def nav_esp_prox(e):
    global esp_page
    esp_page += 1
    lblEspPage.value = f'Página {esp_page + 1}'
    atualizar_feed_esp_publico()

btnEspAnt.on_click(nav_esp_ant); btnEspProx.on_click(nav_esp_prox)

def on_limpar_esp(e=None):
    esp_id.value = 0; esp_nome.value = ''; esp_rua.value = ''; esp_num.value = ''; esp_bairro.value = ''
    tab_esp_admin.selection = []
    on_con_esp()

def on_ins_esp(e=None):
    s = Session()
    try:
        if s.query(EspacoCultural).filter_by(nome=esp_nome.value).first():
            pn.state.notifications.error('Nome existe!')
            return
        s.add(EspacoCultural(nome=esp_nome.value, rua=esp_rua.value, numero=esp_num.value, bairro=esp_bairro.value))
        s.commit(); pn.state.notifications.success('Inserido!'); on_con_esp()
    finally: s.close()

def on_att_esp(e=None):
    if not esp_id.value: return
    s = Session()
    try:
        if o:=s.get(EspacoCultural, esp_id.value):
            changed = (o.nome != esp_nome.value or o.rua != esp_rua.value or o.numero != str(esp_num.value or o.bairro != esp_bairro.value))
            if not changed:
                pn.state.notifications.warning('Nenhuma alteração')
                return
            o.nome=esp_nome.value; o.bairro=esp_bairro.value; o.rua=esp_rua.value; o.bairro=esp_bairro.value; o.numero=esp_num.value; o.bairro=esp_bairro.value
            s.commit(); pn.state.notifications.success('Ok'); on_con_esp()
    finally: s.close()

def on_del_esp(e=None):
    s = Session()
    try: 
        if o:=s.get(EspacoCultural, esp_id.value): s.delete(o); s.commit(); pn.state.notifications.success('Del'); on_con_esp()
    finally: s.close()

btnConEsp.on_click(on_con_esp); btnLimparEsp.on_click(on_limpar_esp)
btnInsEsp.on_click(on_ins_esp); btnAttEsp.on_click(on_att_esp); btnDelEsp.on_click(on_del_esp)

on_con_esp()
tab_espacos = pn.Row(col_esp_main, tab_esp_admin, col_feed_esp_nav, name='Espaços')


In [9]:
# --- 9. GESTÃO DE USUÁRIOS (Admin/Gerente) ---
user_id = pn.widgets.IntInput(name='ID', disabled=True, width=80)
user_nome = pn.widgets.TextInput(name='Nome')
user_email = pn.widgets.TextInput(name='Email')
user_cpf = pn.widgets.TextInput(name='CPF/RG')
user_pass = pn.widgets.PasswordInput(name='Senha (Deixe vazio para manter)')
user_role_select = pn.widgets.Select(name='Papel', options=['COMUM'])
user_cpf.param.watch(format_cpf_live, 'value')

btnConUser = pn.widgets.Button(name='Consultar', button_type='primary')
btnInsUser = pn.widgets.Button(name='Criar Usuário', button_type='success')
btnAttUser = pn.widgets.Button(name='Atualizar', button_type='warning')
btnDelUser = pn.widgets.Button(name='Excluir', button_type='danger')
btnLimparUser = pn.widgets.Button(name='Limpar', button_type='default')

tab_users = pn.widgets.Tabulator(pagination='remote', page_size=20, layout='fit_columns', selectable=1, show_index=False, disabled=True)

col_crud_user = pn.Column('### Gerenciar Usuários', user_id, user_nome, user_email, user_cpf, user_pass, user_role_select,
                          pn.Row(btnConUser, btnLimparUser, btnInsUser, btnAttUser, btnDelUser))

def on_con_users(e=None):
    if not CURRENT_USER: return
    try:
        my_role = str(CURRENT_USER.get('role', '')).upper()
        where_clause = ""
        if 'GERENTE' in my_role:
            where_clause = "WHERE r.nome = 'COMUM'"
        elif 'ADMIN' in my_role:
            where_clause = "WHERE r.nome IN ('GERENTE', 'COMUM')"
        
        query = f"""
            SELECT u.id_usuario, u.nome, u.email, u.cpf_rg, r.nome as role 
            FROM usuario u 
            JOIN role r ON u.id_papel = r.id_papel 
            {where_clause}
            ORDER BY u.id_usuario
        """
        tab_users.value = pd.read_sql_query(query, cnx)
    except Exception as ex: print(f'Erro users: {ex}')

def get_role_id_by_name(rname):
    s = Session(); r = s.query(Role).filter(Role.nome.ilike(f'%{rname}%')).first(); s.close()
    return r.id_papel if r else 2 

def on_sel_user(event):
    if not event.new: return
    row = tab_users.value.iloc[event.new[0]]
    user_id.value = int(row['id_usuario'])
    user_nome.value = str(row['nome'])
    user_email.value = str(row['email'])
    user_cpf.value = str(row['cpf_rg'])
    user_pass.value = ''
    user_role_select.value = str(row['role']).upper()
tab_users.param.watch(on_sel_user, 'selection')

def on_ins_user(e=None):
    s = Session()
    try:
        if s.query(Usuario).filter(or_(Usuario.email==user_email.value, Usuario.cpf_rg==user_cpf.value)).first():
            pn.state.notifications.error('Email ou CPF já existe')
            return
        rid = get_role_id_by_name(user_role_select.value)
        s.add(Usuario(nome=user_nome.value, email=user_email.value, cpf_rg=user_cpf.value, senha_hash=user_pass.value, id_papel=rid))
        s.commit(); pn.state.notifications.success('Usuário criado!'); on_con_users()
    except Exception as x: pn.state.notifications.error(f'Erro: {x}')
    finally: s.close()

def on_att_user(e=None):
    if not user_id.value: return
    s = Session()
    try:
        u = s.get(Usuario, user_id.value)
        if u:
            # DETECT CHANGES
            new_role_id = get_role_id_by_name(user_role_select.value)
            changed = (u.nome != user_nome.value or 
                       u.email != user_email.value or 
                       u.cpf_rg != user_cpf.value or 
                       u.id_papel != new_role_id or 
                       user_pass.value != '')
            
            if not changed:
                pn.state.notifications.warning('Nenhuma alteração detectada')
                return

            u.nome = user_nome.value
            u.email = user_email.value
            u.cpf_rg = user_cpf.value
            if user_pass.value: 
                u.senha_hash = user_pass.value
            u.id_papel = new_role_id
            s.commit(); pn.state.notifications.success('Usuário atualizado!'); on_con_users()
    except Exception as x: pn.state.notifications.error(f'Erro: {x}')
    finally: s.close()

def on_del_user(e=None):
    s = Session()
    try:
        if user_id.value == CURRENT_USER.get('id'):
            pn.state.notifications.error('Não pode se excluir!')
            return
        if o:=s.get(Usuario, user_id.value): s.delete(o); s.commit(); pn.state.notifications.success('Excluído'); on_con_users()
    finally: s.close()

def on_limpar_user(e=None):
    user_id.value = 0
    user_nome.value = ''
    user_email.value = ''
    user_cpf.value = ''
    user_pass.value = ''
    tab_users.selection = []
    on_con_users()

btnConUser.on_click(on_con_users)
btnInsUser.on_click(on_ins_user); btnDelUser.on_click(on_del_user)
btnAttUser.on_click(on_att_user); btnLimparUser.on_click(on_limpar_user)
on_con_users()
tab_usuarios = pn.Row(col_crud_user, tab_users, name='Usuários')


In [ ]:
# --- 9.5 GESTÃO DE DENÚNCIAS (Admin) ---

# 1. WIDGETS
# --- Filtros ---
den_filtro_ev = pn.widgets.TextInput(name='Evento', placeholder='Nome do evento...')
den_filtro_user = pn.widgets.TextInput(name='Usuário', placeholder='Nome do usuário...')
den_filtro_desc = pn.widgets.TextInput(name='Descrição', placeholder='Palavra chave...')
den_filtro_motivo = pn.widgets.TextInput(name='Motivo', placeholder='Trecho do motivo...')
den_filtro_status = pn.widgets.Select(name='Status', options=['', 'aberta', 'em análise', 'fechada', 'rejeitada'], value='')
den_filtro_data = pn.widgets.DatePicker(name='Data')

btn_consultar_den = pn.widgets.Button(name='🔎 Consultar', button_type='primary', width=130, align='end')

# --- Edição (AGORA NO TOPO) ---
lbl_acao = pn.widgets.StaticText(value='<b>Ação na Seleção:</b>', align='end', margin=(5, 5, 10, 20))
den_edit_status = pn.widgets.Select(name='Novo Status', options=['aberta', 'em análise', 'fechada', 'rejeitada'], width=150, align='end')
btn_salvar_status = pn.widgets.Button(name='💾 Salvar', button_type='success', width=100, align='end')

# --- Detalhes (Apenas Leitura) ---
den_detalhe_motivo = pn.widgets.StaticText(name='Motivo da Denúncia', value='Selecione uma linha...')
den_detalhe_texto = pn.widgets.StaticText(name='Descrição Detalhada', value='Selecione uma linha...')


# 2. TABELA

tab_denuncias_admin = pn.widgets.Tabulator(
    pagination='remote', 
    page_size=10, 
    layout='fit_columns',
    sizing_mode='stretch_width', 
    theme='bootstrap4',
    selectable=1,     
    disabled=True     
)


# 3. LÓGICA


# --- Seleção ---
def on_sel_denuncia(event):
    if not event.new: return 
    try:
        row = tab_denuncias_admin.value.iloc[event.new[0]]
        
        # Preenche detalhes
        den_detalhe_motivo.value = str(row['motivo'])
        den_detalhe_texto.value = str(row['Detalhes']) 
        
        # Atualiza o dropdown de edição lá em cima
        status_atual = str(row['status'])
        if status_atual in den_edit_status.options:
            den_edit_status.value = status_atual
            
    except Exception as e:
        print(f"Erro ao selecionar: {e}")

tab_denuncias_admin.param.watch(on_sel_denuncia, 'selection')

# --- Salvar Status ---
def on_salvar_status(e=None):
    if not tab_denuncias_admin.selection:
        pn.state.notifications.warning('Selecione uma denúncia na tabela primeiro!')
        return
    
    s = Session()
    try:
        row_idx = tab_denuncias_admin.selection[0]
        row = tab_denuncias_admin.value.iloc[row_idx]
        id_selecionado = int(row['id_denuncia'])
        
        denuncia = s.get(Denuncia, id_selecionado)
        if denuncia:
            denuncia.status = den_edit_status.value
            s.commit()
            pn.state.notifications.success(f'Status salvo: {den_edit_status.value}')
            buscar_denuncias() # Recarrega a tabela
        else:
            pn.state.notifications.error('Erro: Denúncia não encontrada.')
            
    except Exception as x:
        pn.state.notifications.error(f'Erro ao salvar: {x}')
    finally:
        s.close()

btn_salvar_status.on_click(on_salvar_status)

# --- Buscar (Com correção de data) ---
def buscar_denuncias(event=None):
    f_ev = den_filtro_ev.value.replace("'", "''")
    f_us = den_filtro_user.value.replace("'", "''")
    f_de = den_filtro_desc.value.replace("'", "''")
    f_mt = den_filtro_motivo.value.replace("'", "''")
    f_st = den_filtro_status.value
    val_data = den_filtro_data.value
    f_dt = str(val_data) if val_data else ''
    
    query = f"""
        SELECT 
            d.id_denuncia,
            e.titulo as "Evento",
            u.nome as "Usuário",
            d.motivo,
            d.descricao as "Detalhes",
            d.status,
            d.data
        FROM usuario_denuncia_evento d
        JOIN usuario u ON d.id_usuario = u.id_usuario
        JOIN evento e ON d.id_evento = e.id_evento
        WHERE 
            ('{f_ev}' = '' OR e.titulo ILIKE '%%{f_ev}%%') AND
            ('{f_us}' = '' OR u.nome ILIKE '%%{f_us}%%') AND
            ('{f_de}' = '' OR d.descricao ILIKE '%%{f_de}%%') AND
            ('{f_mt}' = '' OR d.motivo ILIKE '%%{f_mt}%%') AND
            ('{f_st}' = '' OR d.status = '{f_st}') AND
            ('{f_dt}' = '' OR TO_CHAR(d.data, 'YYYY-MM-DD') = '{f_dt}')
        ORDER BY d.data DESC
    """
    try:
        df = pd.read_sql_query(query, cnx)
        tab_denuncias_admin.value = df
        
        den_detalhe_motivo.value = 'Selecione uma linha...'
        den_detalhe_texto.value = 'Selecione uma linha...'
        
        if df.empty and event is not None: 
            pn.state.notifications.warning('Nenhuma denúncia encontrada.')
            
    except Exception as e:
        pn.state.notifications.error(f'Erro consulta: {e}')

btn_consultar_den.on_click(buscar_denuncias)
buscar_denuncias() 

# 4. LAYOUT (AJUSTADO)
layout_denuncias_col = pn.Column(
    '### 🚨 Central de Denúncias',
    
    # --- PAINEL DE CONTROLE ---
    pn.Column(
        # Linha 1: Filtros Texto
        pn.Row(den_filtro_ev, den_filtro_user, den_filtro_desc),
        # Linha 2: Filtros Específicos
        pn.Row(den_filtro_motivo, den_filtro_status, den_filtro_data),
        
        pn.layout.Divider(), 
        
        # Linha 3: BARRA DE AÇÕES (Melhorada)
        pn.Row(
            # Lado Esquerdo: Busca
            btn_consultar_den,
            
            # Mágica: Spacer sem largura definida empurra os lados opostos
            pn.Spacer(),
            
            # Lado Direito: Ações (Agrupados para o texto não ficar perdido)
            pn.Row(
                lbl_acao,
                den_edit_status, 
                btn_salvar_status,
                align='center' # Alinha o texto verticalmente com o botão
            ),
            sizing_mode='stretch_width' # Ocupa a largura total para o Spacer funcionar
        ),
        
        styles={'background': '#fdfdfd', 'padding': '15px', 'border': '1px solid #eee', 'border-radius': '8px'}
    ),
    
    pn.layout.Divider(),
    tab_denuncias_admin,
    pn.layout.Divider(),
    
    # --- DETALHES ---
    '#### Detalhes da Seleção:',
    pn.Row(
        pn.Column(den_detalhe_motivo, styles={'background': '#f8f9fa', 'padding': '10px', 'border-radius': '5px', 'border': '1px solid #ddd'}, margin=(0, 10)),
        pn.Column(den_detalhe_texto, styles={'background': '#f8f9fa', 'padding': '10px', 'border-radius': '5px', 'border': '1px solid #ddd'})
    )
)

tab_denuncias = pn.Row(layout_denuncias_col, name='Denúncias')

In [ ]:
# --- 10. ARTISTAS ---

# Widgets
art_id = pn.widgets.IntInput(name='ID', disabled=True, width=80)
art_nome = pn.widgets.TextInput(name='Nome')
art_cpf = pn.widgets.TextInput(name='CPF/RG')
art_email = pn.widgets.TextInput(name='Email')
art_tel = pn.widgets.TextInput(name='Telefone')
art_desc = pn.widgets.TextAreaInput(name='Descrição', height=100)
art_cpf.param.watch(format_cpf_live, 'value')
art_tel.param.watch(format_tel_live, 'value')

# Buttons
btnConArt = pn.widgets.Button(name='Consultar', button_type='primary')
btnLimparArt = pn.widgets.Button(name='Limpar', button_type='default')
btnInsArt = pn.widgets.Button(name='Inserir', button_type='success')
btnAttArt = pn.widgets.Button(name='Atualizar', button_type='warning')
btnDelArt = pn.widgets.Button(name='Excluir', button_type='danger')

# Layout
col_inputs_art = pn.Column('### Gestão de Artistas', 
                           art_id, art_nome, art_cpf, 
                           pn.Row(art_email, art_tel), 
                           art_desc,
                           pn.Row(btnConArt, btnLimparArt))

row_crud_art_btns = pn.Row(btnInsArt, btnAttArt, btnDelArt)
col_art_main = pn.Column(col_inputs_art, row_crud_art_btns)

# Table
tab_art_admin = pn.widgets.Tabulator(pagination='remote', page_size=20, layout='fit_columns', 
                                     selectable=1, show_index=False, disabled=True,
                                     sizing_mode='stretch_width', theme='bootstrap4')

# Logic
def carregar_form_art(row):
    try:
        art_id.value = int(row['id_artista'])
        art_nome.value = str(row['nome'])
        art_cpf.value = str(row['cpf_rg']) if row['cpf_rg'] else ''
        art_email.value = str(row['email']) if row['email'] else ''
        art_tel.value = str(row['telefone']) if row['telefone'] else ''
        art_desc.value = str(row['descricao']) if row['descricao'] else ''
    except Exception as e: print(f'Error: {e}')

def on_sel_art(event):
    if event.new:
         try:
             row = tab_art_admin.value.iloc[event.new[0]]
             carregar_form_art(row)
         except Exception as e: print(f'Error: {e}')
tab_art_admin.param.watch(on_sel_art, 'selection')

def fetch_artistas(limit=None, offset=0):
    n = art_nome.value.replace("'", "''")
    query = f"SELECT * FROM artista WHERE ('{n}'='' OR nome ILIKE '%%{n}%%') ORDER BY id_artista DESC"
    if limit:
        query += f" LIMIT {limit} OFFSET {offset} "
    return pd.read_sql_query(query, cnx)

def on_con_art(e=None):
    try:
        tab_art_admin.value = fetch_artistas()
    except Exception as x: pn.state.notifications.error(f'{x}')

def on_limpar_art(e=None):
    art_id.value = 0; art_nome.value = ''; art_cpf.value = ''
    art_email.value = ''; art_tel.value = ''; art_desc.value = ''
    tab_art_admin.selection = []
    on_con_art()

def on_ins_art(e=None):
    if not art_nome.value: return
    s = Session()
    try:
        # Check Duplicates CPF/Email
        if s.query(Artista).filter((Artista.cpf_rg==art_cpf.value) | (Artista.email==art_email.value)).first():
             pn.state.notifications.error('CPF ou Email duplicado!')
             return

        new_art = Artista(nome=art_nome.value, cpf_rg=art_cpf.value,
                          email=art_email.value, telefone=art_tel.value,
                          descricao=art_desc.value)
        s.add(new_art); s.commit(); pn.state.notifications.success('Inserido!'); on_con_art()
    except Exception as x: pn.state.notifications.error(f'{x}')
    finally: s.close()

def on_att_art(e=None):
    if not art_id.value: return
    s = Session()
    try:
        a = s.get(Artista, art_id.value)
        if a:
             a.nome = art_nome.value
             a.cpf_rg = art_cpf.value
             a.email = art_email.value
             a.telefone = art_tel.value
             a.descricao = art_desc.value
             s.commit(); pn.state.notifications.success('Atualizado!'); on_con_art()
    except Exception as x: pn.state.notifications.error(f'{x}')
    finally: s.close()

def on_del_art(e=None):
    s = Session()
    try:
        if a:=s.get(Artista, art_id.value): s.delete(a); s.commit(); pn.state.notifications.success('Deletado'); on_con_art()
    finally: s.close()

btnConArt.on_click(on_con_art); btnLimparArt.on_click(on_limpar_art)
btnInsArt.on_click(on_ins_art); btnAttArt.on_click(on_att_art); btnDelArt.on_click(on_del_art)

# Initial Load
# on_con_art()
tab_artistas = pn.Row(col_art_main, tab_art_admin, name='Artistas')


In [ ]:
# --- 10. APP PRINCIPAL (Login + Cadastro + Visitante) ---

header_user = pn.pane.Markdown('')
btnHeaderLogin = pn.widgets.Button(name='Login', button_type='primary', width=120)
btnHeaderLogout = pn.widgets.Button(name='Logout', button_type='default', width=120, visible=False)

tabs_main = pn.Tabs(dynamic=True)

# -------------------------
#  LOGIN / CADASTRO
# -------------------------

# --- Entrar ---
l_email = pn.widgets.TextInput(name='Email')
l_pass = pn.widgets.PasswordInput(name='Senha')
l_btn = pn.widgets.Button(name='Entrar', button_type='primary')
l_msg = pn.pane.Alert('', visible=False, alert_type='danger')

def _do_login(_=None):
    ok = False
    try:
        ok = tentar_login(l_email.value, l_pass.value)
    except Exception as e:
        l_msg.object = str(e)
        l_msg.visible = True
        return

    if ok:
        l_msg.visible = False
        atualizar_interface()
        try: on_con_users()
        except Exception as e: print(f'Error: {e}')
        root.clear(); root.append(app_view)
        pn.state.notifications.success(f'Bem-vindo, {CURRENT_USER["nome"]}!')
    else:
        l_msg.object = 'Email ou senha inválidos.'
        l_msg.visible = True

l_btn.on_click(_do_login)

login_pane = pn.Column(
    pn.pane.Markdown("## Entrar"),
    l_email, l_pass,
    pn.Row(l_btn),
    l_msg,
    width=420
)

# --- Cadastrar (sempre COMUM) ---
c_nome = pn.widgets.TextInput(name='Nome')
c_email = pn.widgets.TextInput(name='Email')
c_cpf = pn.widgets.TextInput(name='CPF/RG')
c_pass = pn.widgets.PasswordInput(name='Senha')
c_pass2 = pn.widgets.PasswordInput(name='Confirmar senha')
c_btn = pn.widgets.Button(name='Cadastrar (Usuário Comum)', button_type='success')
c_msg = pn.pane.Alert('', visible=False, alert_type='danger')


c_cpf.param.watch(format_cpf_live, 'value')

def _do_cadastro(_=None):
    try:
        if not c_nome.value.strip():
            raise ValueError("Informe o nome.")
        if "@" not in c_email.value:
            raise ValueError("Email inválido.")
        if c_pass.value != c_pass2.value:
            raise ValueError("As senhas não conferem.")

        ok = cadastrar_usuario_comum(
            c_nome.value, c_email.value, c_cpf.value, c_pass.value
        )

        if ok:
            c_msg.visible = False
            atualizar_interface()
            try: on_con_users()
            except Exception as e: print(f'Error: {e}')
            root.clear(); root.append(app_view)
            pn.state.notifications.success(f'Cadastro ok! Bem-vindo, {CURRENT_USER["nome"]}!')
        else:
            raise RuntimeError("Cadastro falhou.")

    except Exception as e:
        c_msg.object = str(e)
        c_msg.visible = True

c_btn.on_click(_do_cadastro)

cadastro_pane = pn.Column(
    pn.pane.Markdown("## Não possui cadastro?"),
    c_nome, c_email, c_cpf,
    c_pass, c_pass2,
    c_btn,
    c_msg,
    width=420
)

# --- Visitante ---
v_info = pn.pane.Alert(
    "Você pode usar como Visitante: ver Eventos e Espaços sem cadastro.\n"
    "Para avaliar/denunciar, faça login como Usuário Comum.",
    alert_type="info",
)
v_btn = pn.widgets.Button(name="Continuar como Visitante", button_type="default")

def _do_visitante(_=None):
    logout()
    atualizar_interface()
    root.clear(); root.append(app_view)

v_btn.on_click(_do_visitante)

visitante_pane = pn.Column(
    pn.pane.Markdown("## Visitante"),
    v_info,
    v_btn,
    width=420
)

login_view = pn.Column(
    pn.Spacer(height=15),
    pn.Row(pn.Spacer(),
           pn.Tabs(
               ("Entrar", login_pane),
               ("Cadastrar", cadastro_pane),
               ("Visitante", visitante_pane),
               dynamic=True
           ),
           pn.Spacer()),
)

# -------------------------
# HEADER ACTIONS
# -------------------------
def go_login(_=None):
    root.clear()
    root.append(login_view)

def do_logout(_=None):
    logout()
    atualizar_interface()
    pn.state.notifications.info("Você saiu do sistema.")

btnHeaderLogin.on_click(go_login)
btnHeaderLogout.on_click(do_logout)

# -------------------------
# APP VIEW
# -------------------------
app_view = pn.Column(
    pn.Row('### Gestão Cultural', pn.Spacer(), header_user, btnHeaderLogin, btnHeaderLogout),
    tabs_main
)

# Inicializa como Visitante (já mostra o app direto)
atualizar_interface()
root = pn.Column(app_view)
root.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'3875da53-c660-4219-bee6-c41b64e670a4': {'version…